In [1]:
import os
import joblib
from tqdm import notebook
from scipy.spatial.transform import Rotation
import numpy as np
import cv2

In [2]:
def set_to_origin(mouse, ref_idx2: list):
    # number of timesteps and coordinates
    nts,ncoords = mouse.shape
    # move tail root to origin
    #mousenorm = mouse
    #all x - x_tail; y- y_tail
    #mousenorm = mouse - np.tile(mouse[:,ref_idx2],(1,(int(ncoords / 2))))
    #the array is build like this: bp1_x,bp1_y, bp2_x, bp_y etc.
    #create an array that is ref_bp2 x, y for all bp
    ref_coords = mouse[:,ref_idx2]
    norm_array = np.tile(ref_coords, (1,int(ncoords/2)))
    mousenorm = mouse - norm_array

    return mousenorm

def shift_from_origin(arr, x_shift, y_shift, inplace = False):
    """Shifts all points by (x_shift, y_shift)"""
    if not inplace:
        shift_arr = arr.copy()
        # Even rows, odd columns -> all y
        shift_arr[:, 1::2] += y_shift
        # Odd rows, even columns -> all x
        shift_arr[:, ::2] += x_shift
        return shift_arr
    else:
        # Even rows, odd columns
        arr[:, 1::2] += x_shift
        # Odd rows, even columns
        arr[:, ::2] += y_shift

def magic_transformer(row, ref_point_index):
    pointsT = np.zeros((row.size//2, 3))
    pointsT[:, :2] = row.reshape(row.size//2, 2)
    ref_v = np.zeros((1, 3))
    ref_v[:, 0] = 1
    r, _ = Rotation.align_vectors(ref_v, pointsT[ref_point_index:ref_point_index+1, :])
    return r.apply(pointsT)[:, :2].flatten()


def conv_2_egocentric(arr,  ref_rot_idxs: list, ref_origin_idx: list):
    """
    Calculates egocentric coordinates for mouse and return array
    :param arr: numpy array with bodypart coords X and Y
    :param ref_rot_idxs: reference bodypart index [idx_x, idx_y] that will be used to calculate rotation matrix for; Results in bp on x-axis
    :param ref_origin_idx: reference bodypart that will be new origin (0,0)
    :return: egocentric array
    """

    mouse_data = arr.copy()
    # set one bodypart to new origin mouse
    mousenorm = set_to_origin(mouse_data,ref_origin_idx)
    #rotate to y-axis
    #convert to bp idx
    ref_idx_bp = ref_rot_idxs[0]//2
    rot_mousenorm = np.apply_along_axis(magic_transformer, 1, mousenorm, ref_idx_bp)

    return rot_mousenorm

def get_outline(order_list):
    keypoint_idx_dict = {'nose': (0,1)
                         , 'ear_left':(2,3)
                         , 'ear_right':(4,5)
                         , 'neck':(6,7)
                         , 'hip_left':(8,9)
                         , 'hip_right':(10,11)
                         , 'tail_base':(12,13)}
    order_idx = np.array([np.array(keypoint_idx_dict[x]) for x in order_list])
    return order_idx.flatten()


def get_outline_array(data, bp_idx):

    return data[:,bp_idx]


def animate_blobs(arr, filename, include_dots = False, center_shift = True, show = False, framerate = 30, resolution = (1500, 1500)):

    #center shift
    if center_shift:
        arr_shifted = shift_from_origin(arr, resolution[0]/2, resolution[1]/2)
    else:
        arr_shifted = arr

    #outline
    order1 = ['nose', 'ear_left', 'neck', 'ear_right']
    order2 = ['neck', 'hip_left', 'tail_base', 'hip_right']
    resident_polygon1 = get_outline_array(arr_shifted, get_outline(order1))
    resident_polygon2 = get_outline_array(arr_shifted, get_outline(order2))

    #14 for intruder
    intruder_polygon1 = get_outline_array(arr_shifted, get_outline(order1)+14)
    intruder_polygon2 = get_outline_array(arr_shifted, get_outline(order2)+14)

    if show:
        scale_percent = 30 # percent of original size
        width = int(resolution[0] * scale_percent / 100)
        height = int(resolution[1] * scale_percent / 100)
        dim = (width, height)

    #colors
    magenta = (255,0,255)
    cyan = (255,255, 0)
    White = (255, 255, 255)
    outline_color = [cyan, cyan,magenta, magenta]

    #create videowriter
    #set video parameters
    codec = cv2.VideoWriter_fourcc(
                *"XVID"
            )  # codec in which we output the videofiles


    #out = cv2.VideoWriter(ouput_path, codec, framerate, resolution)
    out = cv2.VideoWriter(filename, cv2.VideoWriter_fourcc(*'X264'), framerate, resolution)

    for idx in np.arange(arr.shape[0]):
        #white background
    #     img = np.ones((resolution[0],resolution[1],3), np.uint8)
    #     img = img * 255
        #black background
        img = np.zeros((resolution[0],resolution[1],3), np.uint8)

        for num, polygon in enumerate([resident_polygon1, resident_polygon2, intruder_polygon1, intruder_polygon2]):
            # #extract bp coordinates
            # bp_coords = polygon.values
            #generate frame precursors from pose info
            frame_coords = polygon[idx]
            #convert into list of tuples (cv2 input)
            #first reshape into [[x1,y1],[x2,y2]] form
            bp_points1 = frame_coords.reshape((int(frame_coords.shape[0]/2), 2))
            #opencv does not take float, so convert poinqts into int for px values
            bp_points1 = bp_points1.astype(int)
            #pts = [(50, 50), (300, 190), (400, 10)]
            bp_points2 = list((map(tuple, bp_points1)))
            #cv2.polylines(img, np.array([pts]), True, RED, 5)
            cv2.fillPoly(img, np.array([bp_points2]), color= outline_color[num])
            if include_dots:
                for bp in bp_points2:
                    cv2.circle(img,bp, 2, White, -1)
        if show:
            # resize image
            resize = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
            cv2.imshow("show", resize)
        # write as video
        out.write(img)
        # exit clauses
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    out.release()

def find_labels(data_train, targets, label_number, sequence2pick = 0):
    if sequence2pick == 0:
        annotation_sequence = targets[:data_train[sequence2pick][num2skip-1:-1:num2skip].shape[0]]

    else:
        ## start from n-1 to n
        annotation_sequence = targets[np.cumsum([data_train[n][num2skip-1:-1:num2skip].shape[0]
                                                     for n in range(sequence2pick)])[-1]:
                                          np.cumsum([data_train[n][num2skip-1:-1:num2skip].shape[0]
                                                     for n in range(sequence2pick)])[-1] +
                                          data_train[sequence2pick][num2skip-1:-1:num2skip].shape[0]]

    label_list = np.argwhere(annotation_sequence == label_number).ravel()

    return annotation_sequence, label_list


def find_bestlabel_examples(data_train, targets,label_number):

    sequence_list = []
    sequence_dict = {}
    for sequence2pick in np.arange(len(data_train)):

        annotation_sequence, label_list = find_labels(data_train,targets, label_number, sequence2pick)
        sequence_dict[sequence2pick] = [annotation_sequence, label_list]
        sequence_list.append((sequence2pick, len(label_list)))

    sequence_length_ordered = sorted(sequence_list,
       key=lambda x: x[1], reverse=True)
    #print(sequence_length_ordered)
    sequence_number = sequence_length_ordered[0][0]
    annotation_sequence, label_list = sequence_dict[sequence_length_ordered[0][0]]

    return annotation_sequence, label_list, sequence_number


def collect_labels(data_train, targets, label_number):
    collection_list = []
    total_labels = 0
    for sequence_num in np.arange(len(data_train)):
        _, l_list = find_labels(data_train, targets, label_number, sequence2pick = sequence_num)
        total_labels += len(l_list)
        collection_list.append(np.array(l_list))
    return collection_list , total_labels

In [3]:
#ego centric conversion params:
intruder_neck_idx = [20,21]
intruder_tailbase_idx = [26,27]
resident_snout = [0,1]
intruder_snout = [14,15]
ego_conv_idxs1 =intruder_neck_idx
ego_conv_idxs2 =intruder_tailbase_idx
# plotting params
framerate = 120
num2skip = int(framerate/10)
#Plotting constants
FRAME_WIDTH_TOP = 1024
FRAME_HEIGHT_TOP = 570
RESIDENT_COLOR = 'cyan'
INTRUDER_COLOR = 'magenta'
PLOT_MOUSE_START_END = [(0, 1), (0, 2), (1, 3), (2, 3), (3, 4),
                        (3, 5), (4, 6), (5, 6), (1, 2)]
class_to_color = {'other': 'gray', 'attack' : 'crimson', 'mount' : 'steelblue',
                   'investigation': 'darkorange', 'Subinvestigate Group 3': 'darkcyan', 'Subinvestigate Group 6': 'purple'}
action_types = ['attack', 'investigation', 'mount', 'other', 'Subinvestigate Group 3', 'Subinvestigate Group 6']
class_to_number = {s: i for i, s in enumerate(action_types)}
number_to_class = {i: s for i, s in enumerate(action_types)}

In [4]:
base_path = '/Users/alexanderhsu/Google Drive/My Drive/2023Data/asoid_manuscript/data2share'
with open(os.path.join(base_path, "data_7bp.sav"), 'rb') as f:
    data_train = joblib.load(f)
with open(os.path.join(base_path, "targets_newinvestigation_8dimumap_split.sav"), 'rb') as f:
    targets = joblib.load(f)
print('{} sequences in training, first has {} frames x {} coords. '.format(len(data_train), *data_train[0].shape))
for action in action_types:
    print(f"{action}: {class_to_number[action]}")
## define an output path, which would contain subdirectories for videos by label
outpath = '/Users/alexanderhsu/Google Drive/My Drive/2023Data/asoid_manuscript/data2share/temp_movies/'

70 sequences in training, first has 4879 frames x 28 coords. 
attack: 0
investigation: 1
mount: 2
other: 3
Subinvestigate Group 3: 4
Subinvestigate Group 6: 5


In [5]:
#find all frames in all sequences for selected class:
for selected_class in range(len(action_types)):
    label_collection, total_labels = collect_labels(data_train, targets, selected_class)
    _, label_list, sequence_number = find_bestlabel_examples(data_train,targets, selected_class)

    print(f"Found {len(label_collection)} sequences with a total of {total_labels} for class {number_to_class[selected_class]} with idx {selected_class}.")
    print(f"Sequence {sequence_number} includes the most annotations of {number_to_class[selected_class]}: {len(label_list)}")
    for sequence_number in notebook.tqdm(range(len(label_collection))):
        transition_idx = np.where(np.diff(label_collection[sequence_number]) != 1)[0]+1
        for i, t in enumerate(transition_idx):
            collection_array = None
            if i == 0:
                for num, l_list in enumerate([label_collection[sequence_number][:t]]):
                    #select only frames from sequence with selected class
                    if collection_array is None:
                        collection_array = data_train[sequence_number][num2skip-1:-1:num2skip][l_list,:]
                    else:
                        collection_array = np.concatenate([collection_array, data_train[sequence_number][l_list,:]], axis = 0)
            else:
                for num, l_list in enumerate([label_collection[sequence_number][transition_idx[i-1]:t]]):
                    #select only frames from sequence with selected class
                    if collection_array is None:
                        collection_array = data_train[sequence_number][num2skip-1:-1:num2skip][l_list,:]
                    else:
                        collection_array = np.concatenate([collection_array, data_train[sequence_number][l_list,:]], axis = 0)

            #align it to egocentric version
            resident_neck_idx = [6,7]
            resident_tail_base_idx = [12, 13]

            intruder_neck = [20,21]
            intruder_tailbase = [26,27]

            resident_snout = [0,1]
            intruder_snout = [14,15]
            ## motion energy works on >= 2 frames
            if (len(collection_array) >= 2):
                class_data_ego = conv_2_egocentric(collection_array,
                                                   ref_rot_idxs = ego_conv_idxs1, ref_origin_idx = ego_conv_idxs2)
                vid_path = os.path.join(outpath, f"{number_to_class[selected_class]}")
                video_name = os.path.join(vid_path, f"ego_{number_to_class[selected_class].replace(' ','_')}_collection_seq{sequence_number}_example{i}.mp4")
                os.makedirs(vid_path, exist_ok=True)
                animate_blobs(class_data_ego, video_name)

Found 70 sequences with a total of 1188 for class attack with idx 0.
Sequence 56 includes the most annotations of attack: 187


  0%|          | 0/70 [00:00<?, ?it/s]

/var/folders/gp/nsk60k_n2cz7kk79_pv33d8r0000gn/T/ipykernel_3939/3696795067.py:36: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  r, _ = Rotation.align_vectors(ref_v, pointsT[ref_point_index:ref_point_index+1, :])
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 a

Found 70 sequences with a total of 8360 for class investigation with idx 1.
Sequence 57 includes the most annotations of investigation: 636


  0%|          | 0/70 [00:00<?, ?it/s]

...

OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
Open

Found 70 sequences with a total of 706 for class Subinvestigate Group 6 with idx 5.
Sequence 8 includes the most annotations of Subinvestigate Group 6: 89


OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'


  0%|          | 0/70 [00:00<?, ?it/s]

OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
OpenCV: FFMPEG: tag 0x34363258/'X264' is not supported with codec id 27 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x31637661/'avc1'
Open